# SQL 入門

> 聯結表格：隨堂練習參考解答

郭耀仁

In [1]:
import sqlite3
import pandas as pd
from sqlFrameCheck import checkAnsQuery
from test_queries.test_queries_03 import extract_test_queries as etq

conn_twelection = sqlite3.connect('twelection.db')
conn_nba = sqlite3.connect('nba.db')

## 隨堂練習：將 `presidential2016` 與 `presidential2020` 三組候選人的得票數以 `UNION` 垂直合併，創建 `year` 變數區分 `number`、`candidates` 與 `total_votes`

In [2]:
ans_query = """
SELECT 2016 AS year,
       number,
       candidates,
       SUM(votes) AS total_votes
  FROM presidential2016
 GROUP BY number
UNION
SELECT 2020 AS year,
       number,
       candidates,
       SUM(votes) AS total_votes
  FROM presidential2020
 GROUP BY number;
"""
# 試跑看看結果
pd.read_sql(ans_query, conn_twelection)

,year,number,candidates,total_votes
0,2016,1,朱立倫/王如玄,3813365
1,2016,2,蔡英文/陳建仁,6894744
2,2016,3,宋楚瑜/徐欣瑩,1576861
3,2020,1,宋楚瑜/余湘,608590
4,2020,2,韓國瑜/張善政,5522119
5,2020,3,蔡英文/賴清德,8170231


## 測資比對

In [3]:
caq = checkAnsQuery(etq('0316'), ans_query, conn_twelection)
caq.run_test()

測資比對正確！


## 隨堂練習：將 `presidential2016` 與 `presidential2020` 三組候選人的得票率以 `UNION` 垂直合併，創建 `year` 變數區分 `number`、`candidates` 與 `votes_percentage`

In [4]:
ans_query = """
SELECT 2016 AS year,
       number,
       candidates,
       CAST(SUM(votes) AS REAL) / CAST((SELECT SUM(votes) FROM presidential2016) AS REAL) AS votes_percentage
  FROM presidential2016
 GROUP BY number
UNION
SELECT 2020 AS year,
       number,
       candidates,
       CAST(SUM(votes) AS REAL) / CAST((SELECT SUM(votes) FROM presidential2020) AS REAL) AS votes_percentage
  FROM presidential2020
 GROUP BY number;
"""
# 試跑看看結果
pd.read_sql(ans_query, conn_twelection)

,year,number,candidates,votes_percentage
0,2016,1,朱立倫/王如玄,0.310409
1,2016,2,蔡英文/陳建仁,0.561234
2,2016,3,宋楚瑜/徐欣瑩,0.128357
3,2020,1,宋楚瑜/余湘,0.042556
4,2020,2,韓國瑜/張善政,0.386137
5,2020,3,蔡英文/賴清德,0.571307


## 測資比對

In [5]:
caq = checkAnsQuery(etq('0317'), ans_query, conn_twelection)
caq.run_test()

測資比對正確！


## 隨堂練習：查詢 `nba.db` 目前湖人隊（Los Angeles Lakers）的球員陣容生涯場均得分（`ppg`）、場均籃板（`rpg`）與場均助攻（`apg`），選擇 `fullName`、`firstName`、`lastName`、`ppg`、`rpg`、`apg` 並以 `firstName` 遞增排序

In [6]:
ans_query = """
SELECT teams.fullName,
       players.firstName,
       players.lastName,
       careerSummaries.ppg,
       careerSummaries.rpg,
       careerSummaries.apg
  FROM players
  JOIN teams
    ON players.teamId = teams.teamId
  JOIN careerSummaries
    ON players.personId = careerSummaries.personId
 WHERE teams.nickname = 'Lakers'
 ORDER BY players.firstName;
"""
# 試跑看看結果
pd.read_sql(ans_query, conn_nba)

,fullName,firstName,lastName,ppg,rpg,apg
0,Los Angeles Lakers,Alex,Caruso,5.7,2.0,2.1
1,Los Angeles Lakers,Anthony,Davis,24.0,10.4,2.2
2,Los Angeles Lakers,Avery,Bradley,11.8,2.9,1.8
3,Los Angeles Lakers,Danny,Green,8.9,3.5,1.6
4,Los Angeles Lakers,Devontae,Cacok,0.0,0.0,0.0
5,Los Angeles Lakers,Dion,Waiters,13.2,2.7,2.8
6,Los Angeles Lakers,Dwight,Howard,16.8,12.3,1.4
7,Los Angeles Lakers,JR,Smith,12.5,3.2,2.1
8,Los Angeles Lakers,JaVale,McGee,7.9,5.1,0.4
9,Los Angeles Lakers,Jared,Dudley,7.5,3.2,1.6


## 測資比對

In [7]:
caq = checkAnsQuery(etq('0320'), ans_query, conn_nba)
caq.run_test()

測資比對正確！


## 隨堂練習：計算 `presidential2020` 韓國瑜/張善政與蔡英文/賴清德這兩組候選人在臺北市 12 個行政區中各自的得票數，選擇 `town`、`Kuo_Cheng` 與 `Ing_Te` 三個變數

In [8]:
ans_query = """
SELECT ing_te.town,
       kuo_cheng.Kuo_Cheng,
       ing_te.Ing_Te
  FROM (SELECT town,
               SUM(votes) AS Kuo_Cheng
          FROM presidential2020
         WHERE county = '臺北市' AND
               number = 2
         GROUP BY town
         ) AS kuo_cheng
  LEFT JOIN (SELECT town,
                    SUM(votes) AS Ing_Te
               FROM presidential2020
              WHERE county = '臺北市' AND
                    number = 3
              GROUP BY town
              ) AS ing_te
    ON kuo_cheng.town = ing_te.town;
"""
# 試跑看看結果
pd.read_sql(ans_query, conn_twelection)

,town,Kuo_Cheng,Ing_Te
0,中山區,56491,79022
1,中正區,41461,48183
2,信義區,62353,70285
3,內湖區,74437,94269
4,北投區,59851,90060
5,南港區,30968,40969
6,士林區,65183,104881
7,大同區,24673,50006
8,大安區,85490,88977
9,文山區,82305,78129


## 測資比對

In [9]:
caq = checkAnsQuery(etq('0318'), ans_query, conn_twelection)
caq.run_test()

測資比對正確！


## 隨堂練習：計算 `presidential2020` 韓國瑜/張善政與蔡英文/賴清德這兩組候選人在臺北市 12 個行政區中各自的得票數，選擇 `town`、`Kuo_Cheng` 與 `Ing_Te` 三個變數，並找出韓國瑜/張善政在哪些行政區得票數較多

In [10]:
ans_query = """
SELECT ing_te.town,
       kuo_cheng.Kuo_Cheng,
       ing_te.Ing_Te
  FROM (SELECT town,
               SUM(votes) AS Kuo_Cheng
          FROM presidential2020
         WHERE county = '臺北市' AND
               number = 2
         GROUP BY town
         ) AS kuo_cheng
  LEFT JOIN (SELECT town,
                    SUM(votes) AS Ing_Te
               FROM presidential2020
              WHERE county = '臺北市' AND
                    number = 3
              GROUP BY town
              ) AS ing_te
    ON kuo_cheng.town = ing_te.town
 WHERE kuo_cheng.Kuo_Cheng > ing_te.Ing_Te;
"""
# 試跑看看結果
pd.read_sql(ans_query, conn_twelection)

,town,Kuo_Cheng,Ing_Te
0,文山區,82305,78129


## 測資比對

In [11]:
caq = checkAnsQuery(etq('0319'), ans_query, conn_twelection)
caq.run_test()

測資比對正確！
